In [3]:
# 使用submitter对api进行访问
from sparksampling import Submitter
from sparksampling.utilities.var import FILE_TYPE_CSV
from sparksampling.utilities.var import SIMPLE_RANDOM_SAMPLING_METHOD
submitter = Submitter()
dataset_uri = 'hdfs://localhost:9000/dataset/titanic/train.csv'
fraction = 0.4

In [4]:
# 提交抽样任务
submit_response = submitter.submit_sampling_simplejob(dataset_uri,
                                              method=SIMPLE_RANDOM_SAMPLING_METHOD,
                                              file_type=FILE_TYPE_CSV,
                                              fraction=fraction,
                                              with_header=True)
job_id = submit_response.job_id
submit_response.to_dict()

request: http://localhost:8000/v1/sampling/simplejob/
2021-04-01 19:32:00,714 DEBUG connectionpool.py[231] Starting new HTTP connection (1): localhost:8000
2021-04-01 19:32:00,745 DEBUG connectionpool.py[461] http://localhost:8000 "POST /v1/sampling/simplejob/ HTTP/1.1" 200 49


{'code': 0, 'msg': '', 'data': {'job_id': 10050}}

In [5]:
# 查询抽样任务
sampling_job_details = submitter.get_sampling_job_details(job_id)
sampled_path = sampling_job_details.sampled_path
print(sampling_job_details.to_dict())
sampled_path

request: http://localhost:8000/v1/sampling/query/job/
2021-04-01 19:32:10,410 DEBUG connectionpool.py[231] Starting new HTTP connection (1): localhost:8000
2021-04-01 19:32:10,416 DEBUG connectionpool.py[461] http://localhost:8000 "POST /v1/sampling/query/job/ HTTP/1.1" 200 462
{'code': 0, 'msg': '', 'data': {'job_id': 10050, 'job_status': 'Succeed', 'msg': 'succeed', 'method': 'Simple Random Sampling', 'start_time': '2021/04/01/ 19:32:01', 'end_time': '2021/04/01 19:32:02', 'simpled_file_path': 'hdfs://localhost:9000/dataset/titanic/train.csv-sampled-1617276720.7170603', 'request_data': "{'path': 'hdfs://localhost:9000/dataset/titanic/train.csv', 'method': 'random', 'type': 'csv', 'with_header': True, 'conf': {'fraction': 0.4}}"}}


'hdfs://localhost:9000/dataset/titanic/train.csv-sampled-1617276720.7170603'

In [6]:
# 简单的读取抽样之后的文件
from pyspark.sql import SparkSession
from sparksampling.config import SPARK_CONF

conf = SPARK_CONF
spark = SparkSession.builder.config(conf=conf).getOrCreate()
df = spark.read.csv(sampled_path, header=True)
df.toPandas()
# 可以在这后面做数据分析，或试试看下面的统计、评估功能

# data analyse here


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.925,None,S
3,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.05,None,S
4,8,0,3,"Palsson, Master. Gosta Leonard",male,2,3,1,349909,21.075,None,S
...,...,...,...,...,...,...,...,...,...,...,...,...
371,881,1,2,"Shelley, Mrs. William (Imanita Parrish Hall)",female,25,0,1,230433,26,None,S
372,885,0,3,"Sutehall, Mr. Henry Jr",male,25,0,0,SOTON/OQ 392076,7.05,None,S
373,886,0,3,"Rice, Mrs. William (Margaret Norton)",female,39,0,5,382652,29.125,None,Q
374,889,0,3,"""Johnston, Miss. Catherine Helen """"Carrie""""""",female,None,1,2,W./C. 6607,23.45,None,S


In [7]:
# 统计原数据集
submitter.get_statistics(path=dataset_uri, file_type=FILE_TYPE_CSV,with_header=True).to_pandas()

request: http://localhost:8000/v1/evaluation/statistics/
2021-04-01 19:33:52,601 DEBUG connectionpool.py[231] Starting new HTTP connection (1): localhost:8000
2021-04-01 19:33:53,704 DEBUG connectionpool.py[461] http://localhost:8000 "POST /v1/evaluation/statistics/ HTTP/1.1" 200 1414


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
summary,,,,,,,,,,,,
count,891,891,891,891,891,714,891,891,891,891,204,889
mean,446.0,0.3838383838383838,2.308641975308642,None,None,29.69911764705882,0.5230078563411896,0.38159371492704824,260318.54916792738,32.2042079685746,None,None
stddev,257.3538420152301,0.48659245426485753,0.8360712409770491,None,None,14.526497332334035,1.1027434322934315,0.8060572211299488,471609.26868834975,49.69342859718089,None,None
min,1,0,1,"""Andersson, Mr. August Edvard (""""Wennerstrom"""")""",female,0.42,0,0,110152,0,A10,C
max,99,1,3,"van Melkebeke, Mr. Philemon",male,9,8,6,WE/P 5735,93.5,T,S


In [8]:
# 统计抽样后的数据集
data = submitter.get_statistics(job_id=job_id, from_sampling=True, file_type=FILE_TYPE_CSV,with_header=True).to_pandas()
data

request: http://localhost:8000/v1/evaluation/statistics/
2021-04-01 19:33:57,520 DEBUG connectionpool.py[231] Starting new HTTP connection (1): localhost:8000
2021-04-01 19:33:57,965 DEBUG connectionpool.py[461] http://localhost:8000 "POST /v1/evaluation/statistics/ HTTP/1.1" 200 1414


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
summary,,,,,,,,,,,,
count,376,376,376,376,376,296,376,376,376,376,102,374
mean,464.2925531914894,0.39095744680851063,2.247340425531915,None,None,29.603918918918918,0.5159574468085106,0.38563829787234044,262423.08921933087,34.09537765957447,None,None
stddev,261.3650413713812,0.4886150640852816,0.8702449316915051,None,None,15.458875203458867,1.0977604539171548,0.7883864480629801,454985.4185076666,45.357145181350575,None,None
min,10,0,1,"""Bowen, Mr. David John """"Dai""""""",female,0.42,0,0,110152,0,A16,C
max,97,1,3,"Williams, Mr. Charles Eugene",male,9,8,6,WE/P 5735,93.5,G6,S


In [9]:
# 提交评估任务
evaluation_job = submitter.submit_evaluation_job(compare_job_id=job_id, file_type=FILE_TYPE_CSV)
print(evaluation_job.to_dict())
evaluation_job_id = evaluation_job.job_id
evaluation_job_id

request: http://localhost:8000/v1/evaluation/job/
2021-04-01 19:34:01,722 DEBUG connectionpool.py[231] Starting new HTTP connection (1): localhost:8000
2021-04-01 19:34:01,743 DEBUG connectionpool.py[461] http://localhost:8000 "POST /v1/evaluation/job/ HTTP/1.1" 200 49
{'code': 0, 'msg': '', 'data': {'job_id': 50023}}


50023

In [10]:
# 查看各属性评估得分
evaluation_job_data = submitter.get_evaluation_job_details(job_id=evaluation_job_id)
df = evaluation_job_data.to_pandas()
df

request: http://localhost:8000/v1/evaluation/query/job/
2021-04-01 19:34:07,985 DEBUG connectionpool.py[231] Starting new HTTP connection (1): localhost:8000
2021-04-01 19:34:07,990 DEBUG connectionpool.py[461] http://localhost:8000 "POST /v1/evaluation/query/job/ HTTP/1.1" 200 2610


,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
count,296,102,374,376,376,376,376,376,376,376,376,376
mean,29.603918918918918,None,None,34.09537765957447,None,0.38563829787234044,464.2925531914894,2.247340425531915,None,0.5159574468085106,0.39095744680851063,262423.08921933087
stddev,15.458875203458867,None,None,45.357145181350575,None,0.7883864480629801,261.3650413713812,0.8702449316915051,None,1.0977604539171548,0.4886150640852816,454985.4185076666
min,0.42,A16,C,0,"""Bowen, Mr. David John """"Dai""""""",0,10,1,female,0,0,110152
max,9,G6,S,93.5,"Williams, Mr. Charles Eugene",6,97,3,male,8,1,WE/P 5735
mean_bias,0.00320544,-1,-1,0.0587243,-1,0.0105992,0.0410147,0.0265531,-1,0.0134805,0.018547,0.00808448
stddev_bias,0.0641846,-1,-1,0.0872607,-1,0.0219225,0.0155863,0.0408741,-1,0.00451871,0.00415668,0.0352492
score,96.6305,-1,-1,92.7007,-1,98.3739,97.1699,96.6286,-1,99.1,98.8648,97.8333


In [11]:
import pandas as pd
df = df.drop(columns='PassengerId',axis=1)
score_list = df.loc['score'].to_list()
while -1 in score_list:
    score_list.remove(-1)
score_list
import numpy as np
np.mean(score_list)

97.1617102391623